<a href="https://colab.research.google.com/github/Risotto04/DATA-ENGINEERING-LAB/blob/main/LAB22/LAB02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
class DBconfig:
  MYSQL_HOST = "db4free.net"
  MYSQL_PORT = 3306
  MYSQL_USER = "week2_de_lab"
  MYSQL_PASSWORD = "week2_de_lab"
  MYSQL_DB = "week2_de_lab"

In [121]:
! pip install pymysql

In [122]:
import pymysql as mysql
connection = mysql.connect(host=DBconfig.MYSQL_HOST,
                           port = DBconfig.MYSQL_PORT,
                           user = DBconfig.MYSQL_USER,
                           password = DBconfig.MYSQL_PASSWORD,
                           db = DBconfig.MYSQL_DB,
                           cursorclass = mysql.cursors.DictCursor)

In [123]:
with connection.cursor() as cs:
  cs.execute("show tables;")
  tables = cs.fetchall()

for t in tables:
  print(t)

{'Tables_in_week2_de_lab': 'customers'}
{'Tables_in_week2_de_lab': 'employees'}
{'Tables_in_week2_de_lab': 'offices'}
{'Tables_in_week2_de_lab': 'orderdetails'}
{'Tables_in_week2_de_lab': 'orders'}
{'Tables_in_week2_de_lab': 'payments'}
{'Tables_in_week2_de_lab': 'productlines'}
{'Tables_in_week2_de_lab': 'products'}


In [124]:
def execute(sql):
  with connection.cursor() as cs:
    cs.execute(sql)
    result = cs.fetchall()

  for d in result:
    print(d)

In [125]:
execute('SELECT customerNumber, customerName, country FROM  customers WHERE country ="UK";')

{'customerNumber': 187, 'customerName': 'AV Stores, Co.', 'country': 'UK'}
{'customerNumber': 201, 'customerName': 'UK Collectables, Ltd.', 'country': 'UK'}
{'customerNumber': 240, 'customerName': 'giftsbymail.co.uk', 'country': 'UK'}
{'customerNumber': 324, 'customerName': 'Stylish Desk Decors, Co.', 'country': 'UK'}
{'customerNumber': 489, 'customerName': 'Double Decker Gift Stores, Ltd', 'country': 'UK'}


In [126]:
# data2 = execute("1")

In [127]:
sql = '''SELECT  products.productName , products.productVendor ,
          orderdetails.priceEach * orderdetails.quantityOrdered as income , orders.orderDate
          FROM orders
          INNER JOIN orderdetails INNER JOIN products
          ON orderdetails.orderNumber=orders.orderNumber AND orderdetails.productCode=products.productCode
          WHERE products.productVendor = 'Exoto Designs' AND MONTH(orders.orderDate) = 2
          ORDER BY orders.orderDate ASC;'''
execute(sql)

In [128]:
sql = '''SELECT count(p.customerNumber) as TotalNumCustomer FROM payments p  INNER JOIN customers c
        ON p.customerNumber = c.customerNumber
        WHERE p.amount > (SELECT sum(amount)/count(amount) FROM payments) '''
execute(sql)

{'TotalNumCustomer': 134}


In [129]:
sql = ''' SELECT firstName, lastName
        FROM employees
        WHERE jobTitle LIKE "%Sales"'''
execute(sql)

{'firstName': 'Mary', 'lastName': 'Patterson'}


In [130]:
import pandas as p

In [134]:
with connection.cursor() as cs:
  cs.execute('SELECT customerNumber, customerName, country FROM  customers WHERE country ="UK";')
  tables = cs.fetchall()

  data = p.DataFrame(tables)
data.set_index('customerNumber')

,customerName,country
customerNumber,,
187,"AV Stores, Co.",UK
201,"UK Collectables, Ltd.",UK
240,giftsbymail.co.uk,UK
324,"Stylish Desk Decors, Co.",UK
489,"Double Decker Gift Stores, Ltd",UK


In [147]:
data = p.read_sql('SELECT * FROM offices', connection)
data.set_index('officeCode')

<ipython-input-147-804fd1ff5c9d>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = p.read_sql('SELECT * FROM offices', connection)


,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
officeCode,,,,,,,,
1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,None,None,France,75017,EMEA
5,Tokyo,+81 33 224 5000,4-1 Kioicho,None,Chiyoda-Ku,Japan,102-8578,Japan
6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,None,Australia,NSW 2010,APAC
7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,None,UK,EC2N 1HN,EMEA


In [169]:
sql = ''' SELECT customerName ,orderNumber
        FROM customers
        INNER JOIN orders
        ON customers.customerNumber = orders.customerNumber
        WHERE customerName = 'Alpha Cognac' '''
custome= p.read_sql(sql, connection)
custome

<ipython-input-169-4aba315efd10>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  custome= p.read_sql(sql, connection)


,customerName,orderNumber
0,Alpha Cognac,10136
1,Alpha Cognac,10178
2,Alpha Cognac,10397


In [170]:
sql = ''' SELECT *  FROM  orders'''
orders = p.read_sql(sql, connection)
data = orders.merge(custome)
data = data[['customerNumber','orderNumber','customerName']]
data.set_index('customerNumber')

<ipython-input-170-9a119ae9cd15>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orders = p.read_sql(sql, connection)


,orderNumber,customerName
customerNumber,,
242,10136,Alpha Cognac
242,10178,Alpha Cognac
242,10397,Alpha Cognac


In [191]:
sql = ''' SELECT *  FROM  payments'''
payments = p.read_sql(sql, connection)
payments['THBamount'] = payments['amount'] *30
payments.set_index(['customerNumber','checkNumber'],inplace=True)
payments

<ipython-input-191-c3facf9d8469>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  payments = p.read_sql(sql, connection)


paymentDate    amount  THBamount
customerNumber checkNumber                                 
103            HQ336336     2004-10-19   6066.78   182003.4
               JM555205     2003-06-05  14571.44   437143.2
               OM314933     2004-12-18   1676.14    50284.2
112            BO864823     2004-12-17  14191.12   425733.6
               HQ55022      2003-06-06  32641.98   979259.4
...                                ...       ...        ...
495            BH167026     2003-12-26  59265.14  1777954.2
               FN155234     2004-05-14   6276.60   188298.0
496            EU531600     2005-05-25  30253.75   907612.5
               MB342426     2003-07-16  32077.44   962323.2
               MN89921      2004-12-31  52166.00  1564980.0

[273 rows x 3 columns]

In [213]:
print(payments.dtypes,end='\n\t\t\U00002B07\n')
payments= payments.astype({'THBamount':'int64'})
print(payments.dtypes)


paymentDate     object
amount         float64
THBamount      float64
dtype: object
		⬇
paymentDate     object
amount         float64
THBamount        int64
dtype: object
